<a href="https://colab.research.google.com/github/PorkPy/LSTM-Force-Predictor/blob/master/joint_data_good_model_31_07_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google Drive
Allow Colab to access your Google Drive for saving models and other things. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Libraries

In [ ]:
% reset -f
from __future__ import print_function
from __future__ import division
import torch
import torch.nn.functional as F
from torch import nn, optim
import sys
import os
import glob
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as plticker
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import random
import time
from datetime import datetime
from subprocess import call
import warnings
#warnings.filterwarnings("ignore")

## Set random seed for numpy and Torch
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Version Data
This block return Python, PyTorch and CUDA versions, plus, the number of GPUs available as well as any prosesses currently running. The gpu data can be copyed to elsewhere in the code, such as in the training loop to check gpu prosesses and memory usage. 

versions used here are:

    Python VERSION: 3.6.9 
    pyTorch VERSION: 1.5.1+cu101
    CUDA VERSION:
        Cuda compilation tools, release 10.1, V10.1.243
        CUDNN VERSION: 7603


In [ ]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print('Available devices ', torch.cuda.device_count())
print('Current cuda device ', torch.cuda.current_device())

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Check CUDA is available

In [ ]:
torch.cuda.is_available()



# Input Model Hyper-parameter settings
---
Here, one can stipulate the model number which is used to make a unique directory in your Google drive, along with sub-directories containing all the testing metrics. 

The hyper-parameters that can be changed here are:
<br>
<br>
**lr** = learning rate.
<br>
**fc** = number of fully connected layers bolted on to the back of the LSTM layers.
<br>
**path** = your desiered path to save and load models in Google Drive. 
<br>
**hidden** = number of hidden LSTM layers.
<br>
**epochs** = number of times to run through the whole training data.
<br>
**seq_length** = sequence length = the number of samples in each slice of a trajectory.
<br>
**feature_num** = number of features or independent variables in the data. 
<br>
<br>
Adjusting the number of features allows users to use a variable number of features or different types of data, such as joint data with 4 joints or Cartesian data with 6 Cartesian positions. 
<br>

The model automatically adjusts itself depending on the number of layers and features required. 

---




In [ ]:
'''              ### MODEL HYPER-PARAMETER SETTINGS ###
-------------------------------------------------------------------------------
Here, the model hyper-parameters can be set, along with the save model path 
and the warm start model parameters path if using a pretrained model. 
'''
torch.manual_seed(42)
model_num  = '501'        ## Unique model number for saving new model.
model_dir  = 'model501'   ## New directory name to save new model to. 
params     = '115_v0'     ## Pretrained model params to load...
warm_start = False        ## ...and if to load them or not.
seq_length = 250          ## The length of the trajectory slice trained on.
epochs     = 101          ## Number of full passes through the whole dataset.
hidden     = 60           ## Number of nodes in the LSTM layers.
lr         = 0.0005       ## Learning rate.
feature_num   =  4        ## 4 features for joint data, 6 features for cartesian data.
fc         = 1            ## Number of fully connected layers. 1 or 2.
random_seed = 0           ## Used to seed the random number generator for reproducibility.
path       = f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/" ## Save directory.

#------------------------------------------------------------------------------
## I increased the batch size and lr by 1 order.

## Functions to fetch hyper-parameters
def model_number():
    return model_num

def load_params():
    return params

def model_directory():
    return model_dir

def get_seq_length():
    return seq_length

def get_epochs():
    return epochs

def get_warm_start():
    return warm_start

def get_hidden():
    return hidden

def get_lr():
    return lr

def get_path():
    return path

def get_features():
    return feature_num

def get_fc():
    return fc

def get_random_seed():
    return random_seed

## Dictionary with which to save paramers.
param = {'Model Num':model_num, 
          'Seq Length': seq_length,
          'Epochs': epochs,
          'Warm Start': (warm_start),
          'Hidden Size': hidden,
          'Learning Rate': lr,
          'features': feature_num, 
          'Num LSTM Layers':2,
          'Num FC Layers':1,
          'Data/Notes': 'full cart data, changed where zero_grad is, loss reporting/saving.'
}

## Create new directory in perent directory to save parameters.
try:
    os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)


## create a pandas data frame of the model parameters and save to csv.
param = pd.DataFrame(param, index=[0])
param.to_csv(path + "lstm_params.csv", index=False)


# Testing Loop
---

This block fetches the testing data and passes it through the model after a set number of training runs.
<br>
For each trajectory in the training data, three metrics are obtained:
<br>
<br>
**MAE** = Mean Absolute Error accross each trajectory, measured in Newtons.
<br>
**RMSD** = Root Mean Sqared Deviation. Similar to MAE, but sensitive to outliers.
<br>
**COV** = Coefficient of Variance = RMSD/mean of the dataset. Useful for comparing performance across different models and datasets.
<br>
<br>
For each trained model, we find the **Grand Mean** or mean of means, **Standard Deviation** and **Max Value** over all the testing trajectories.
<br>

**PDF** or Probability Density Function and **Gaussian Distribution** plots of each metric are then created to give an intuative view of the metrics used.

---


In [ ]:
def lighten_color(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

def tests(model_name):
    
    ## fetch parameters
    seq_len = get_seq_length()
    model_name = model_name
    model_dir = model_directory()
    path = get_path()
    features_num = get_features()

    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

    stats_list = []
    pdf = PdfPages(path + f"testing_traj_pics_{model_name}.pdf")
    fig = plt.figure()

    
    #model = model.to(device)
    #print(model)
    #print("testing weights", model.linear1.weight.data) # Check weights are being updated.

    #model.reset_hidden_state()
    for traj in range(len(test_batches)):
        model.reset_hidden_state()
        whole_traj = []
        whole_true = []

        for start_seq in range(int(1000/seq_length)): 
            start_seqx = start_seq*seq_length ## get the next sequence start position
            #model.reset_hidden_state()

            Xtest, ytest = get_test_batch(traj, start_seqx)
            model.eval()
            with torch.no_grad():
                
                x = iter(Xtest)
                test_seq = Xtest[0].reshape(-1,seq_len,features_num)#.reshape(1,200,4) # input first sequence from trajectory/batch
                preds = [] # create a list to store predictions.
                for i in range(len(Xtest)): # for each sequence i in the trajectory,
                    y_test_pred = model(test_seq).to(device)# send sequence to model,
                    pred = torch.flatten(y_test_pred).cpu() # reshape the model output,
                    preds.append(np.asarray(pred)) # and append to the list of predictions - preds.
                    new_seq = next(x).reshape(-1,seq_len,features_num)#.reshape(1,200,4) # Change sequence to the next one in the list.
                    test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_len, -1) # change sequence to a torch Tensor
            whole_traj.append(preds)
            whole_true.append(ytest)
        whole_true = np.array(whole_true).reshape(-1,)
        ## rescale the output predictions
        preds = target_scaler.inverse_transform(whole_traj).reshape(-1,3)
        ## Vector summation - the vector sum of the 3 output predictions
        force_vec = np.sqrt((preds[:,0]**2)+(preds[:,1]**2)+(preds[:,2]**2))
        
        ytest = whole_true
        preds = force_vec ## reset name to comply with existing code.
        #display(force_vec)
        
        #Mean Absolute Error
        MAE_list = []
        for i,j in zip(preds, ytest):
            error = np.abs(i-j)
            MAE_list.append(error)
        MAE = float("{:.3f}".format(np.mean(MAE_list)))
        #print("MAE","{:.3f}".format(MAE),'N')

        # Coefficient of Variance
        mean = np.mean(data.iloc[:,-1]) # mean of all dependent variables.
        cov_list = []
        for i,j in zip(preds, ytest):
            sq_dev = (i-j)**2
            cov_list.append(sq_dev)    
        MSD = np.mean(cov_list) # mean square deviation
        RMSD = np.sqrt(MSD) # root mean square deviation
        cov = RMSD/mean # coefficient of variance
        RMSD = float("{:.3f}".format(RMSD))
        cov =  float("{:.3f}".format(cov))
        #print("COV:","{:.3f}".format(cov))
        
    
        my_dict = {'Trajectory':traj,
                'MAE': MAE, 
                'RMSD':RMSD,
                'cov': cov, # Used to normalise the RMSD accross all the data
        }
        stats_list.append(my_dict)

        # Plot forces
        predicted_cases = preds
        true_cases = ytest
        # Add title and axis names
        plt.title(f'Force Trajectory {traj}');
        plt.xlabel('Sample num');
        plt.ylabel('Force (N)');
        plt.tight_layout();
        plt.grid(True)
        plt.ylim(-1, 70)
        plt.plot(true_cases, color=lighten_color('b', 1.7), linewidth=3.0, label='Real Force');
        plt.plot(predicted_cases, color=lighten_color('r', 1.0), linewidth=1.0, label='Predicted Force');
        plt.legend(loc=2, prop={'size': 6})

        # save the current figure
        pdf.savefig(fig);
        # destroy the current figure
        plt.clf()

    pdf.close()
    stats_list = pd.DataFrame(stats_list)
    return stats_list

###############################################################################

def stats(stats_list2, model_name):
    
 
    ## Get the mean of MAE, RMSD and cov.
    mean_list = {
                'MAE' :float("{:.3f}".format(np.mean(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.mean(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.mean(stats_list2['cov'])))
    }
    ## Get the std-dev of MAE, RMSD and cov.
    std_dev = {
                'MAE' :float("{:.3f}".format(np.std(stats_list2['MAE']))),
                'RMSD':float("{:.3f}".format(np.std(stats_list2['RMSD']))),
                'cov' :float("{:.3f}".format(np.std(stats_list2['cov'])))
    }
    ## Get the max value of MAE, RMSD and cov.
    max_list = {
                'MAE' :float(stats_list2['MAE'].max()),
                'RMSD':float(stats_list2['RMSD'].max()),
                'cov' :float(stats_list2['cov'].max())
    }
    ## append above dicts to stats_list2.
    stats_list2 = stats_list2.append(mean_list, ignore_index=True).fillna('Grand Mean')
    stats_list2 = stats_list2.append(std_dev, ignore_index=True).fillna('Standard Dev')
    stats_list2 = stats_list2.append(max_list, ignore_index=True).fillna('Max Value')

    #display(stats_list2)
    path = get_path()
    
    ## Create new directory in perent directory
    path = path + f"{model_name}/"
    model_name = model_name

    ## save stats_list as .csv in same directory as trajectory plots.
    stats_list2.to_csv(path + f"lstm_model_metrics_{model_name}.csv", index=False)
    return stats_list2

###############################################################################

## Get a Gaussian Distribution of the MAE, RMSD and cov.
def gauss_plot(stats_list2, name, error_type, num):
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"

    error = error_type ## Either; MAE, RMSD or cov.
    pdf = PdfPages(path + f"gauss_pic_{error}.pdf")
    fig = plt.figure()
    
    # define constants
    mu = np.mean(stats_list2.iloc[:-3,num]) 
    sigma = np.sqrt(np.var(stats_list2.iloc[:-3,num]))
    x1 = np.min(stats_list2.iloc[:-3,num])
    x2 = np.max(stats_list2.iloc[:-3,num])
    

    # calculate the z-transform
    z1 = ( x1 - mu ) / sigma
    z2 = ( x2 - mu ) / sigma

    x = np.arange(z1, z2, 0.001) # range of x in spec
    x_all = np.arange(-10, 10, 0.001) # entire range of x, both in and out of spec
    # mean = 0, stddev = 1, since Z-transform was calculated
    y = norm.pdf(x,0,1);
    y2 = norm.pdf(x_all,0,1);

    # build the plot
    fig, ax = plt.subplots(figsize=(9,6));
    #plt.style.use('fivethirtyeight');
    ax.plot(x_all,y2);

    ax.fill_between(x,y,0, alpha=0.3, color='b');
    ax.fill_between(x_all,y2,0, alpha=0.1);
    ax.set_xlim([-4,4]);
    ax.set_xlabel('# of Standard Deviations Outside the Mean');
    ax.set_yticklabels([]);
    ax.set_title(f'{model_name} {error} Std Dev');

    plt.savefig('normal_curve.png', dpi=72, bbox_inches='tight');
    plt.grid(True);
    plt.tight_layout();
    #plt.show()
    # save the current figure
    pdf.savefig(fig);
    ## destroy the current figure
    plt.clf()

    # close the object
    pdf.close()

###############################################################################

## Get a PDF of the MAE, RMSD and cov.
def prob_dist(stats_list2, name, error_type, num):    
    model_name = name
    model_dir = model_directory()
    path = get_path()
    path = path + f"{model_name}/"


    error = error_type
    pdf = PdfPages(path + f"prob_dist_pic_{error}.pdf")
    fig = plt.figure()

    import seaborn as sns
    sns.distplot(stats_list2.iloc[:-3,num], color="darkslategrey");
    plt.xlabel("Force [newtons]", labelpad=14);
    plt.ylabel("Probability of Occurence", labelpad=14);
    plt.title(f"Probability Distribution of {error}", fontsize=20);
    plt.grid(True);
    plt.tight_layout();

    #plt.show()
    # save the current figure
    pdf.savefig(fig);
    # destroy the current figure
    plt.clf()
    plt.close('all') ## added this due to runtime warning, more than 20 figs open
    # close the object
    pdf.close()


# Test Controller
This block automates the execution of the testing code above by first calling tests() which generates the testing trajectory plots, but also calculates the MAE, RMSD and COV for each trajectory. The product of which is saved as a dictionary and returned.


In [ ]:
def test_runner(name):   
    stats_df = tests(name) # Run tests on testing data and save generated plots to Google Drive
    stats(stats_df, name) # Record stats and save to Google Drive
    for i in range(1,4): # 1 to 3 = the colunms in the stats_list DataFrame
        if i ==1:
            error_type = 'MAE' # mean absolur error
        elif i == 2:
            error_type = 'RMSE' # root mean squared error
        elif i == 3:
            error_type = 'cov' # coefficient of variance

        prob_dist(stats_df, name, error_type, i) # Gen prob_dist and save to GD
        
        gauss_plot(stats_df, name, error_type, i) # Gen Gauss plots and save to GD
    print("Done")

# LSTM Network
---
The network consists of two LSTM layers with sixty nodes each, plus one fully connected output layer; however, these hyper-parameters can be adjusted in the hyper-parameter section above. The network is able to automatically adjust the number of LSTM layers, nodes and fully connected layers between one and three when stipulated in the hyper-parameter block. 
<br>
The reason for having adjustable fully connected layers here was because it was fount that the network could sometimes produce better models with more layers depending on the input data; however, the nature of the data has changed significantly since the start of this project, but it is still interesting to compare the performance when increasing the network complxity. 
<br>
A lot of experiments were carried out on all the available activation functions in PyTorch and 'Leaky relu' was found to perform the best for this task.

---

In [ ]:
class ForcePredictor(nn.Module):

    def __init__(self, n_features, n_hidden, seq_len, n_layers=2, ignore_zero=True):
        super(ForcePredictor, self).__init__()

        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            print("Running on the GPU")
        else:
            device = torch.device("cpu")
            print("Running on CPU")

        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=0.5)
        
        
        fc = get_fc() ## get num of FC layers

        if fc == 1:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=3)
            
        elif fc == 2:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear2 = nn.Linear(in_features=60, out_features=3)

        elif fc == 3:
            self.linear1 = nn.Linear(in_features=n_hidden, out_features=60)
            self.linear2 = nn.Linear(in_features=60, out_features=60)
            self.linear3 = nn.Linear(in_features=60, out_features=3)
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device)
        )

    ## Forward Function.
    
    def forward(self, sequences):

        '''                    ## Forward Method ##
        -----------------------------------------------------------------------
        This Method takes the input and passes it through each of the network 
        layers.
        The number of fully connected layers the input gets passed through is
        dependent on the number stipulated in the network hyper-parameters at
        the beginning of the notebook.
        ----------------------------------------------------------------------- 
        '''
        
        fc = get_fc() ## get num of FC layers

        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences), self.seq_len, -1),self.hidden)
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]

        if fc == 1:
            y_pred = self.linear1(last_time_step)

        if fc == 2:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = self.linear2(y_pred)

        if fc == 3:
            y_pred = F.leaky_relu(self.linear1(last_time_step))
            y_pred = F.leaky_relu(self.linear2(y_pred))
            y_pred = self.linear3(y_pred)

       

        return y_pred

# Docstring Tester
---
This is just a little tester block to make sure the docstrings throughout the code are working as expected.

---

In [ ]:
my_object = ForcePredictor(_,_,_)
print(my_object.forward.__doc__)

# Training Loop
---
This is where the magic happens!


In [ ]:
def train_model(model):
    '''                   ## training Loop ##
    ---------------------------------------------------------------------------

    ---------------------------------------------------------------------------
    '''
    lr = get_lr()
    loss_fn = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#0.0007 
    print("learning rate =", lr) 
    num_epochs = get_epochs() #1600 #600
    path = get_path()
    seq_length = get_seq_length()

    #--------------------------------------------------------------------------
    '''                   ## Get Model Parameters ##
    ---------------------------------------------------------------------------
    If you want to restart training from an earlier model, first - it should be
    stipulated in the hyper-parameter setting at the beginning of the notebook
    by setting 'warm_start' = true, and adding the path to the saved model 
    location.


    '''
    start_epoch = 0
    warm_start = get_warm_start()
    if warm_start == True:      
        params = load_params() # model num and version num: 4_v100.
        PATH = f"/content/drive/My Drive/PhD/PhD/lstm/model_params{params}.pt"     
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda()
        start_epoch = checkpoint['epoch']
        loss = checkpoint['loss']
    device = torch.device("cuda")
    model = model.to(device)

    train_hist = np.zeros(num_epochs + start_epoch)
    test_hist = np.zeros(num_epochs + start_epoch)

    print(model)
    print("Starting Weights", model.linear1.weight.data) # Check weights are being updated.
    start_weights = model.linear1.weight.data.clone()

    ## create new lists to save the losses over the whole training phase.
    tot_losses = []
    tot_test_losses = []
    losses = []
    test_losses = []

    ## Set time with which to reference elapsed time.
    s1 = time.strftime('%H:%M:%S')

    '''                   ## Start of Training Loop
    ---------------------------------------------------------------------------
    start_epoch = 0 unless using pre-trained model.
    num_epochs is stipulated in model hyper-parameter settings.
    
    '''

    for t in range(start_epoch, num_epochs):

        ## Get elasped time for each epoch.
        s2 = time.strftime('%H:%M:%S')
        FMT = '%H:%M:%S'
        tdelta = datetime.strptime(s2, FMT) - datetime.strptime(s1, FMT)
        print("\n Time Elapsed", tdelta)
       
        for j in range(len(batches)): #-----------------------------------------## for each traj in training data.
            if t % 10 != 0: 
                print("\r", f"Epoch:{t} Batch:{j}", end="") #-----------## print epoch & batch for a visual reference during training. 

            model.reset_hidden_state() #----------------------------------------## reset hidden state for the start of each trajectory
            #losses = [] #-------------------------------------------------------## reset sum of losses for each trajectoy. 
            optimizer.zero_grad() #---------------------------------------------## maybe keeping the gradiants over the whole trajectoy is a good idea??
            ## Fetch n samples from each trajectory to train on before updatining network.
            for start_seq in range(int(1000/seq_length)): #---------------------## 1000 samples divided by seq length returns num slices.
                train_data, train_labels, _, _ = get_batches(j, start_seq*seq_length) ## dataloader- featch each slice of the traj at a time.
                y_pred = model(train_data) #------------------------------------## make prediction on a slice/batch.
                loss = loss_fn(y_pred.float(), train_labels) #------------------## loss for each slice/batch.
                losses.append(loss.item()) #------------------------------------## append for all slices/batches == 1 trajectory.
                
                with torch.no_grad(): #-----------------------------------------## for some reason, the model breaks if this code block isn't here??? the model looses the map.
                    random_pred = model(train_data) #---------------------------## Accessing the model seems to have some effect on the loss???
                        
                train_hist[t] = loss.item()
                #optimizer.zero_grad() #----------------------------------------## moved zerograd to start of each trajectory. ### having zero_grad here produces strange peaks at start of trajectory
                loss.backward() #-----------------------------------------------## Calculate the gradients of the loss wrt the network weights.
                #model = model.to(device) #-------------------------------------## put model on GPU. ### shouldn't need this as already declared. 
                optimizer.step() #----------------------------------------------## Update network weights.
            traj_loss = np.mean(losses) #---------------------------------------## average loss over each trajectory


            model.reset_hidden_state() #----------------------------------------# Seperate training and testing hidden states. 
            #test_losses = []
            for start_seq in range(int(1000/seq_length)): 

                #start_seqx = start_seq*seq_length ## get the next sequence start position
                _, _, test_data, test_labels = get_batches(j, start_seq*seq_length)
                with torch.no_grad():
                    y_test_pred = model(test_data)
                    test_loss = loss_fn(y_test_pred.float(), test_labels) ## loss for each batch
                test_hist[t] = test_loss.item()
                test_losses.append(test_loss.item()) ## append loss for each trajectory
            traj_test_loss = np.mean(test_losses) ## average test loss over each tarjectory of n samples


                ## The loss will look small (<1) but that's because we are not de-scaling the output. 
            if t % 10 == 0: 
                print(f'Epoch {t} {j} train loss: {traj_loss} test loss: {traj_test_loss}') ## display losses for each trajectory every 10 epochs.
                
        #tot_losses.append(loss.item()) ## save loss for each trajectory.
        #tot_test_losses.append(test_loss.item())

        ## Periodically save model and show training and testing loss
        if t % 10 == 0:
            print('Saving model', '\n')
            model_num = model_number()
            model_save_name = f'model_params{model_num}_v{t}.pt'
            torch.save({
                'epoch': num_epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss':loss,},
                f"/content/drive/My Drive/PhD/PhD/lstm/{model_save_name}" 
            )

            ## Show the training and testing losses during execution.
            #from matplotlib.pyplot import figure
            #figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
            #figure(figsize=(20,4))
            #plt.plot(losses, label='Training Loss');
            #plt.plot(test_losses, label='Testing Loss');
            #plt.legend();
            #plt.show()
            #plt.pause(0.01)
            mean_loss = np.mean(losses)
            mean_test_loss = np.mean(test_losses)
            print("\n Average Loss")
            print(mean_loss)
            print("\n Average Test Loss")
            print(mean_test_loss,'\n')
            diff = mean_test_loss - mean_loss
            print("Difference Between Training and Testing Losses")
            print(diff.item(),'\n')
            tot_losses.append(mean_loss)
            tot_test_losses.append(mean_test_loss)
        

            name = f'model{model_num}_v{t}'
            test_runner(name)
            model.train() ## just in case it was left in .eval() mode.

            model_dir = model_directory()

            pdf = PdfPages(f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/loss.pdf")
            fig = plt.figure();

            plt.plot(tot_losses, label='Training Loss');
            plt.plot(tot_test_losses, label='Testing Loss');
            plt.xlabel("100 epochs", labelpad=14);
            plt.ylabel("Loss", labelpad=14);
            plt.title(f"Training and Testing Losses {model_dir}", fontsize=20);
            plt.grid(True);
            plt.tight_layout();
            plt.legend();

            # save the current figure
            pdf.savefig(fig);

            figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
            plt.show()
            plt.pause(0.01)

            # destroy the current figure
            plt.clf()
            # close the object
            pdf.close()

        losses = []
        test_losses = []

    return model.eval(), train_hist, test_hist, optimizer, t, loss_fn

# Import Data
---
This section did automatically selects which dataset to download based on the feature number selection made at the start. 
<br>
However, it was descovered that the model can in fact utilise two or more cleaned datasets, picking feature columns as necessary, assuming the datasets are of the same original raw data.
<br>
For example, The Cartesian feature model uses the X, Y, Z, Rx, ry and Rz features of the Cartesian dataset, together with the Fx, Fy and Fz target features of the joints dataset.
<br>
Of course, these datasets could be combined and a lot of processed datasets based on the original raw data have been created and added to the gitbub repositiory; however, i think it's interesting to see how different datasets can be combined.
<br>
4 features for the joint data and 6 features for the Cartesian data. 

---

In [ ]:
## Get num
feature_num = get_features()
url = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/cart_data_plus_rotation.csv'

url2 = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/4_joints_3_force_1_forceVec.csv'

data = pd.read_csv(url)
data2 = pd.read_csv(url2)
main_seq = data

In [ ]:
display(data2)

# Data Selection and Scaling
---
This block selects the desired features and targets from the datasets above and scales each section. The features and targets are scaled individually so that the output from the model can be de-scaled easily. 

---

In [ ]:
features = data2.iloc[:,3:7]
feature_scaler = StandardScaler()
features = feature_scaler.fit_transform(features)

targets = data2.iloc[:,:3]
target_scaler = StandardScaler()
targets = target_scaler.fit_transform(targets)

force_vec = pd.DataFrame(data2.iloc[:,-1])

features = pd.DataFrame(features)
targets = pd.DataFrame(targets)
data = pd.concat([targets, features, force_vec], axis=1)

data.columns = [['Fx', 'Fy', 'Fz', data2.columns[3], data2.columns[4], data2.columns[5], data2.columns[6], 'force vec']]
print(data.columns)
display(data)

# Create Training and Testing Data Batches
---
This block takes the whole dataset and divides it up into individual trajectories that are 1000 samples each.
<br>
As the data were collected by performing different kineasthetic trajectories through the taskspace, and each trajectory was performed ten times; the trajectories are shuffled to remove corrolations between them. 
<br>
Later, you will see how the trajectories themselves are further split into slices, and the model is trained on one slice at a time. Typical slice sizes have been 500, 250 and 125.
<br>
Because of this extra slicing, one may ask; why not just divide the dataset into 500, 250 or 125 sample trajectories to begine with?
<br>
We need to shuffle the trajectories to break up any corrolation between them, but doing this to all the slices would destroy the collolarion between slices belonging to a each trajectory. 
<br>
Every batch of slices needs to be kelpt together during traing because they are still used as full trajectories by the model during training by not resetting the hidden states of the LSTM layers until all slices of a particular trajectory have been seen. 
<br>
This means, that although the model has its parameters updated after seeing each slice, the history of previous slices still influence the training until the full trajectoy has passed through and the hidden states get reset.
<br><br>
Due to the current size of the dataset, The training set consists of sixty trajectories.
<br>
This leave a remander of eleven trajectories to test with; therefore, testing and validation sets use the same eleven trajectories; however,
in order to compare training and testing loss as the model learns, the number of testing trajectory is inflated by copying them until they are the same length.
<br>
To overcome the problem of such a small testing set, cross validation is used to compare models and get an average loss overall. 
The cross validation is simply initiated by changing the random seed at the beginning of the notebook which reshuffels the data, resulting in largly different training and testing trajectories. 


---

In [ ]:
n=1000  ## num samples per trajectory/sequence.
batchesx = [data[i:i + n] for i in range(0, len(data), n)] ## a list comprehension to build the data batches.
print(len(batchesx))

random.seed(get_random_seed())
random.shuffle(batchesx)


batches = batchesx[:60] ## Training batches up to the 60th sequence/trajectory.
val_batches = batchesx[60:] ## Validation batches starting from the 60th sequence.

## Append extra validation batches to even the number training and validation batches.
## This is because the training loop performs a validation test on each iteration
## and so always needs something to validate against.  
while len(val_batches) < len(batches): 
    for i in batchesx[60:]:
        val_batches.append(i)
random.shuffle(val_batches)

test_batches = batchesx[60:] ## Testing batches, same as validation batches, without the appendages.
print(len(batches), len(val_batches), len(test_batches))

# Create Sequence Batches
---
It turns out that this sequencial data problem is more similar to natural language processing rather than stock pridiction because we have set sequences of data rather than continuous data, and because we need to make predictions righ from the start of each sequence not just at the end of them.

---

<br>
Natural language prosessing or nlp, has the problem of needing to input sequences of differing lengths (because words and sentences have different lengths) into an recurrant network that only accepts fixed length vectors. 
<br>
To Overcome this, npl engineers use 'zero padding' which fills up a word or sentence vector with zeros until it is the correct size for the network. 
<br>
This work is a little different but the zero padding technique helps a lot but allowing use to input every timestep of a trajectory from beginning to end while making predictions on every one. 
<br>
To makes things a little mopre clear- the first datapoint of a trajectory is just that, one single datapoint, a sequence consisting of one timestep, but the final datapoint is part of a sequence containing a history of the previous 999 timesteps. 
<br>
In order to fit these and all the other datapoints into the network,
they need to be zeropadded to length 1000.
The first sequence to be fed through the network will consist of one single datapoint and '999*num_features' zeros. 
<br>
This means, that despite only having one single datapoint at the beginning of the trajectory, our LSTM-based model can still make a valid prediction of the force at that point.  

---

In [ ]:
def get_batches(batch_num, start_seq):  
    
    seq_size = get_seq_length() # 1000 = full trajectories
    features_num = get_features()

    # random.seed(batch_num)
    # random.shuffle(batches) # ive turned this off to test new cleaned data

    # Randomise the fetching of new data to break the corrolation of training.
    #print(batch_num)
    #print(type(batches[batch_num])) 
    data = batches[batch_num].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]
    ################################################

    X_train = []
    X_test = []
    
    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)


    targets = data.iloc[:,:3]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,3)

    ## Create Zero-Padded Training Batches ##
    for i in range(len(features)):           
        
        np.random.seed(42)
       
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_train.append(temp)
    y_train = targets
    #--------------------------------------------------------------------------
    
    ## Validation Batches
    data = val_batches[batch_num].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]

    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)


    targets = data.iloc[:,:3]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,3)
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)
    y_test = targets
##############################################################################
    ## Change data to cuda tensors to use on gpu.
    X_train = torch.cuda.FloatTensor(X_train) 
    y_train = torch.cuda.FloatTensor(y_train)
    X_test = torch.cuda.FloatTensor(X_test)
    y_test = torch.cuda.FloatTensor(y_test)
    
    del targets, features, data ## clear large volumn variables from mem
    
    #print(data)
    #print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    return(X_train, y_train, X_test, y_test)

# Test Batches
---



In [ ]:
def get_test_batch(batch_number, start_seq):
    
    seq_size = get_seq_length() ## 1000 for testing 50-100 for training
    features_num = get_features()

    X_test = []

    data = test_batches[batch_number].reset_index(drop=True)
    data= data[start_seq:seq_size+start_seq]

    if features_num == 4:
        features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    else:
        features = data[['x', 'y', 'z', 'Rx', 'Ry', 'Rz']]
    features = np.asarray(features)
    y_test = data.iloc[:,-1]
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, features_num))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)

    
    X_test = torch.cuda.FloatTensor(X_test)
    return(X_test, y_test)
  

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")


In [ ]:
%%time

tot_losses = []
tot_test_losses = []

seq_length = get_seq_length() # when using zero padding, this seq_length is a bit redundent but still has to match the zero's size.

model = ForcePredictor(
      n_features=get_features(), 
      n_hidden= get_hidden(), #32, #64
      seq_len=seq_length, 
      n_layers=2
    )

model, train_hist, test_hist, optimizer, epochs, loss = train_model(model)



In [ ]:
print("Saving model")
model_num = model_number()
model_save_name = f'model_params{model_num}_vlast.pt'
torch.save({
   # 'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss':loss,},
    f"/content/drive/My Drive/PhD/PhD/lstm/{model_save_name}" 
)

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k');
figure(figsize=(20,4));
plt.plot(train_hist, label="Training Loss");
plt.plot(test_hist, label="Testing loss");
plt.title("Training and Testing Loss for Every Epoch",fontsize=30)
plt.grid(True);
plt.tight_layout();
plt.legend(loc=2, prop={'size': 15})
plt.show()

RL Controller Predictor

# MAE, Std-Dev and Max Values Plot
---
This code looks at all the metrics collected and plots the results for easy reference. 
<br>
This removes the need to go through each model's testing trajectories looking for the best fitting model.

---

In [ ]:
def find_file(mode_dir):
    
    means = []
    std_dev = []
    max = []
    PATH =  path = f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/*/*.csv"
    for file in glob.glob(PATH):
        w = pd.read_csv(file, low_memory=False)
        x = w.iloc[-3,1]
        y = w.iloc[-2,1]
        z = w.iloc[-1,1]
        
        means.append(x)
        std_dev.append(y)
        max.append(z)
    return means, std_dev, max
    


In [ ]:
model_num = model_number()

means, std_dev, max = find_file(f'model{model_num}')

figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k');
figure(figsize=(20,4));
fig = plt.figure();
plt.plot(means, label='MAE');
plt.plot(std_dev, label='std Dev');
plt.plot(max, label='Max Values');
plt.xlabel("x10 epochs", labelpad=14);
plt.ylabel("Error (N)", labelpad=14);
plt.title(f"Testing Error {model_dir}", fontsize=20);

plt.grid(True);
plt.legend();
plt.tight_layout();
plt.show();


In [ ]:
pdf = PdfPages(f"/content/drive/My Drive/PhD/PhD/lstm/{model_dir}/error.pdf")

# save the current figure
pdf.savefig(fig);
# destroy the current figure
plt.clf()
# close the object
pdf.close()
